### 9.4 Richardson-Extrapolation zum Limes

In [ ]:
import numpy as np
from scripts.Interpolation import neville, differenz_einseitig_1, differenz_zentral_1, differenz_zentral_2

#### Beispiel 9.26 (Extrapolation des einseitigen Differenzenquotienten)

Es sei $f(x)=\tanh(x)$. Wir wollen die Ableitung von $f$ an der Stelle $x_0=1/2$ auswerten. Der exakte Wert ist $f'(1/2) \approx 0.786448$. Hierzu nehmen wir den einseitigen Differenzenquotienten und werten diesen zu den Schrittweiten $h=2^{-1}, 2^{-2}, 2^{-3}, 2^{-4}$
aus:

In [ ]:
data = [[0.5**i, differenz_einseitig_1(np.tanh, 0.5, 0.5**i)] for i in range(1, 5)]
data = np.array(data)
print(data)

Formal können wir nun ein Interpolationspolynom $p(h)$ durch die Punkte $h_i=2^{-i}$ und $a_i=a(h_i)$ legen und dieses an der Stelle $h=0$ auswerten. Dies geschieht effizient mit dem Neville-Schema zum Punkt $h=0$:

In [ ]:
vals = neville(data, 0)
print(vals)

Damit ergeben sich die Fehler

In [ ]:
print(np.triu(vals - 1 / np.cosh(0.5)**2))

Durch Extrapolation kann die Genauigkeit also wirklich verbessert werden.

#### Beispiel 9.28 (Extrapolation des zentralen Differenzenquotienten)
Wir wiederholen das vorherige Beispiel mit dem zentralen Differenzenquotienten

In [ ]:
data = [[0.5**i, differenz_zentral_1(np.tanh, 0.5, 0.5**i)] for i in range(1, 5)]
data = np.array(data)
print(data)

In [ ]:
vals = neville(data, 0)
print(vals)

Damit ergeben sich die Fehler

In [ ]:
print(np.triu(vals - 1 / np.cosh(0.5)**2))

Das modifizierte Neville-Schema zur Richardson-Extrapolation implementieren wir mit

In [ ]:
def extrapolate(data, q):
    n = data.shape[0]
    h = data[:, 0]
    a = np.zeros((n, n))
    a[:, 0] = data[:, 1]
    
    for i in range(1, n):
        for k in range(1, i + 1):
            a[i, k] = a[i, k - 1] + (a[i, k - 1] - a[i - 1, k - 1]) / ((h[i - k] / h[i])**q - 1)
    return a

#### Beispiel 9.31 (Extrapolation des zentralen Differenzenquotienten)
Wir wenden nun die Richardson-Extrapolation auf den zentralen Differenzenquotienten mit Ordnung $q=2$ an. 

In [ ]:
vals = extrapolate(data, 2)
print(vals)

Damit ergeben sich die Fehler

In [ ]:
print(np.tril(vals - 1 / np.cosh(0.5)**2))

#### Beispiel 9.32 (Extrapolation mit verschiedenen Fehlerordnungen) 
Wir betrachten nun $$f(x) = -e^{1-\cos(\pi x)}$$ und approximieren $f''(1)$ mit dem zentralen Differenzenquotienten.

In [ ]:
data = [[0.5**i, differenz_zentral_2(lambda x: - np.exp(1 - np.cos(np.pi * x)), 1, 0.5**i)]
        for i in range(8)]
f2_ex = np.pi**2 * np.exp(2)
data = np.array(data)
print(data)

Wenden wir eine einfache lineare Extrapolation darauf an, erhalten wir

In [ ]:
vals = extrapolate(data, 1)[:,:3]
print(vals)

Wir sehen, dass der erste Schritt der Extrapolation die Konvergenzordnung nicht verbessert:

In [ ]:
err = np.abs(np.tril(vals - f2_ex))
out = np.zeros((err.shape[0], 6))
out[:, 0] = err[:, 0]
out[:, 2] = err[:, 1]
out[:, 4] = err[:, 2]
for j in range(3):
    for i in range(1 + j, err.shape[0]):
        out[i, 2 * j + 1] = (np.log(err[i - 1][j]) - np.log(err[i][j])) / np.log(2)
print(out.astype(np.half))

Wenden wir allerdings die richtige quadratische Extrapolation an

In [ ]:
vals = extrapolate(data, 2)[:,:3]
print(vals)

Wir sehen, dass wir nach einem Extrapolationsschritt bereits Konvergenz vierter Ordnun erhalten und mit zwei Schritten Konvergenz sechster Ordnung erreichen.

In [ ]:
err = np.abs(np.tril(vals - f2_ex))
out = np.zeros((err.shape[0], 6))
out[:, 0] = err[:, 0]
out[:, 2] = err[:, 1]
out[:, 4] = err[:, 2]
for j in range(3):
    for i in range(1 + j, err.shape[0]):
        out[i, 2 * j + 1] = (np.log(err[i - 1][j]) - np.log(err[i][j])) / np.log(2)
print(out.astype(np.half))